In [1]:
from GENIE3 import *

In [11]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
# import SAUCIE
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

sys.path.append(os.getcwd())

In [6]:
y = np.transpose(np.load('../previous_imputations/DS6_45.npy'))

In [7]:
x = np.transpose(np.load('../previous_imputations/DS6_clean.npy'))

In [9]:
target_file = '../SERGIO/data_sets/De-noised_1200G_9T_300cPerT_6_DS3/Interaction_cID_6.txt'

In [91]:
def gt_benchmark(virtual_imputation):
    gt_temp = np.zeros_like(virtual_imputation)
    f = open(target_file,'r')
    Lines = f.readlines()
    f.close()
    for j in tqdm(range(len(Lines))):
        line = Lines[j]
        line_list = line.split(',')
        target_index = int(float(line_list[0]))
        num_regs = int(float(line_list[1]))
        # skip if gene is not present in filtered dataset
        if target_index >= gt_temp.shape[0]:
            continue
        else:
            for i in range(num_regs):
                reg_index = int(float(line_list[i+2]))
                gt_temp[reg_index,target_index] = 1  
    return gt_temp

In [64]:
# Load SAUCIE imputed data
y_hat_saucie = np.load('../previous_imputations/yhat_saucie.npy')
y_hat_saucie2 = np.load('../SAUCIE/yhat_saucie.npy')

# Load scScope imputed data
y_hat_scscope = np.load('../previous_imputations/yhat_scScope.npy')
y_hat_scscope2 = np.load('../scScope/yhat_scScope.npy')

# Load DeepImpute imputed data
y_hat_deepimp = np.load('../previous_imputations/yhat_deepimp.npy')
y_hat_deepimp2 = np.load('../deepimpute/yhat_deepimp.npy')

# Load MAGIC imputed data
y_hat_magic_t2 = np.load('../MAGIC/yhat_MAGIC_t_2.npy')
y_hat_magic_t7 = np.load('../MAGIC/yhat_MAGIC_t_7.npy')
y_hat_magic_t_auto = np.load('../MAGIC/yhat_MAGIC_t_auto.npy')

In [65]:
print("----------")
print(f"number of zeros in previous SAUCIE: {y_hat_saucie.size - np.count_nonzero(y_hat_saucie)}")
print(f"number of zeros in current SAUCIE:  {y_hat_saucie2.size - np.count_nonzero(y_hat_saucie2)}")
print("----------")
print(f"number of zeros in previous scscope: {y_hat_scscope.size - np.count_nonzero(y_hat_scscope)}")
print(f"number of zeros in current scscope:  {y_hat_scscope2.size - np.count_nonzero(y_hat_scscope2)}")
print("----------")
print(f"number of zeros in previous deepimpute: {y_hat_deepimp.size - np.count_nonzero(y_hat_deepimp)}")
print(f"number of zeros in current deepimpute:  {y_hat_deepimp2.size - np.count_nonzero(y_hat_deepimp2)}")
print("----------")
print(f"number of zeros in MAGIC t=2:       {y_hat_magic_t2.size - np.count_nonzero(y_hat_magic_t2)}")
print(f"number of zeros in MAGIC t=7:       {y_hat_magic_t7.size - np.count_nonzero(y_hat_magic_t7)}")
print(f"number of zeros in MAGIC t=default: {y_hat_magic_t_auto.size - np.count_nonzero(y_hat_magic_t_auto)}")

----------
number of zeros in previous SAUCIE: 0
number of zeros in current SAUCIE:  1
----------
number of zeros in previous scscope: 452511
number of zeros in current scscope:  333501
----------
number of zeros in previous deepimpute: 1183960
number of zeros in current deepimpute:  1183210
----------
number of zeros in MAGIC t=2:       71732
number of zeros in MAGIC t=7:       0
number of zeros in MAGIC t=default: 0


### Run GENIE3 on SAUCIE Data

Previous Data

In [ ]:
VIM_SAUCIE = GENIE3(y_hat_saucie,nthreads=12,ntrees=100)

In [ ]:
gt = gt_benchmark(VIM_SAUCIE)

In [14]:
roc_auc_score(gt.flatten(),VIM_SAUCIE.flatten())

0.4538860037246973

Current Data

In [ ]:
VIM_SAUCIE = GENIE3(y_hat_saucie2,nthreads=12,ntrees=100)

In [ ]:
roc_auc_score(gt.flatten(),VIM_SAUCIE.flatten())

### Run GENIE3 on scScope Data

Previous Data

In [ ]:
VIM_scScope = GENIE3(y_hat_scscope,nthreads=12,ntrees=100)

In [ ]:
roc_auc_score(gt.flatten(),VIM_scScope.flatten())

Current Data

In [ ]:
VIM_scScope = GENIE3(y_hat_scscope2,nthreads=12,ntrees=100)

In [ ]:
roc_auc_score(gt.flatten(),VIM_scScope.flatten())

### Run GENIE3 on DeepImpute Data

Previous Data

In [50]:
VIM_DeepImpute = GENIE3(y_hat_deepimp,nthreads=12,ntrees=100)

Tree method: RF
K: sqrt
Number of trees: 100


running jobs on 12 threads
Elapsed time: 1482.39 seconds


In [53]:
roc_auc_score(gt.flatten(),VIM_DeepImpute.flatten())

0.32298617893241455

Current Data

In [58]:
VIM_DeepImpute = GENIE3(y_hat_deepimp2,nthreads=12,ntrees=100)

Tree method: RF
K: sqrt
Number of trees: 100


running jobs on 12 threads
Elapsed time: 1550.74 seconds


In [59]:
roc_auc_score(gt.flatten(),VIM_DeepImpute.flatten())

0.5095581685268773

### Run GENIE3 on MAGIC Data

In [ ]:
VIM_MAGIC = GENIE3(y_hat_magic_t2, nthreads=12, ntrees=100)

In [ ]:
gt = gt_benchmark(VIM_MAGIC)

In [ ]:
roc_auc_score(gt.flatten(), VIM_MAGIC.flatten())

In [88]:
VIM_MAGIC = GENIE3(y_hat_magic_t7, nthreads=12, ntrees=100)

Tree method: RF
K: sqrt
Number of trees: 100


running jobs on 12 threads
Elapsed time: 1034.69 seconds


In [92]:
gt = gt_benchmark(VIM_MAGIC)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 1133/1133 [00:00<00:00, 227691.36it/s]


In [93]:
roc_auc_score(gt.flatten(), VIM_MAGIC.flatten())

0.5037603476085601

In [66]:
VIM_MAGIC = GENIE3(y_hat_magic_t_auto, nthreads=12, ntrees=100)

Tree method: RF
K: sqrt
Number of trees: 100


running jobs on 12 threads
Elapsed time: 967.02 seconds


In [86]:
gt = gt_benchmark(VIM_MAGIC)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 1133/1133 [00:00<00:00, 214388.99it/s]


In [87]:
roc_auc_score(gt.flatten(), VIM_MAGIC.flatten())

0.4990104796650178

### Benchmark: GENIE3 on Clean Data

Previous Data

In [60]:
x = np.transpose(np.load('../previous_imputations/DS6_clean.npy'))

In [ ]:
VIM_CLEAN = GENIE3(x, nthreads=12, ntrees=100)

In [ ]:
roc_auc_score(gt.flatten(), VIM_CLEAN.flatten())

Current Data

In [ ]:
x_curr = np.transpose(np.load('../SERGIO/imputation_data/DS6_clean.npy'))

In [ ]:
VIM_CLEAN = GENIE3(x_curr, nthreads=12, ntrees=100)

In [ ]:
roc_auc_score(gt.flatten(), VIM_CLEAN.flatten())

### Benchmark: GENIE3 on Noisy Data

Previous Data

In [ ]:
y = np.transpose(np.load('../previous_imputations/DS6_45.npy'))

In [ ]:
VIM_NOISY = GENIE3(y, nthreads=12, ntrees=100)

In [ ]:
roc_auc_score(gt.flatten(), VIM_NOISY.flatten())

Current Data

In [ ]:
y_curr = np.transpose(np.load('../SERGIO/imputation_data/DS6_45.npy'))

In [ ]:
VIM_NOISY = GENIE3(y_curr, nthreads=12, ntrees=100)

In [ ]:
roc_auc_score(gt.flatten(), VIM_NOISY.flatten())